# Deep Learning Model to Classify Lake-Effect Mode
This code will use several years of human-labelled lake-effect mode data (reflectivity) over the Tug Hill to train a deep learning model to classify lake-effect mode.

# Modules

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
import os, sys
import numpy as np
import pandas as pd
import xarray as xr
import dask.array as da
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.patheffects as PathEffects
import scipy.ndimage
import matplotlib.patches as patches
sys.modules['__main__'].__file__ = 'ipython'
from netCDF4 import Dataset
from glob import glob

# Create Cluster

In [ ]:
from dask.distributed import Client, LocalCluster
c = LocalCluster(n_workers=40, threads_per_worker=1)
client = Client(c)
client

# Read in netcdf radar files and csv files with labels

In [ ]:
%time ds_rad = xr.open_mfdataset('/glade/scratch/tomg/tug_radar_netcdf_daily/*', parallel = True,\
                                 coords="minimal", data_vars="minimal", compat="minimal", chunks={'time': 1})




#%time ds_rad = xr.open_mfdataset('/glade/scratch/tomg/tug_radar_netcdf_monthly/*', parallel = True, concat_dim="time",\
#                  data_vars='minimal', coords='minimal', compat='override')

In [ ]:
#Convert time into YYYYMMDDHHMM
year = ds_rad['time.year'].values
month = ds_rad['time.month'].values
day = ds_rad['time.day'].values
hour = ds_rad['time.hour'].values
minute = ds_rad['time.minute'].values

rad_time = np.zeros(len(year))
for i in range(len(year)):
    rad_time[i] = np.int("{:04d}{:02d}{:02d}{:02d}{:02d}".format(year[i], month[i], day[i], hour[i], minute[i]))
rad_time = rad_time.astype(int)

# Read in CSV files with Labels

In [ ]:
####################### Labels #################################
# SHORE = 0
# MISC = 1
# OROG = 2
# HYBRID = 3
# MCV = 4
# LLAP1 = 5
# BROAD = 6

#Read in Labels
labels_raw = pd.read_excel('/glade/scratch/tomg/tug_mode_labels/Morphology_Type_ScatEdit.xlsx')
labels = np.squeeze(np.array(labels_raw), axis = 1)

labels_num = np.zeros(len(labels))-1
for i in range(len(labels)):

    if labels[i] == 'SHORE':
        labels_num[i] = 0
    elif labels[i] == 'MISC':
        labels_num[i] = 1
    elif labels[i] == 'OROG':
        labels_num[i] = 2
    elif labels[i] == 'HYBRID':
        labels_num[i] = 3
    elif labels[i] == 'MCV':
        labels_num[i] = 4
    elif labels[i] == 'LLAP1':
        labels_num[i] = 5
    elif labels[i] == 'BROAD':
        labels_num[i] = 6
    
    
####################### Label Times ###############################
#Read in start and end times
labels_start_raw = pd.read_excel('/glade/scratch/tomg/tug_mode_labels/Morphology_Start_ScatEdit.xlsx')
labels_end_raw = pd.read_excel('/glade/scratch/tomg/tug_mode_labels/Morphology_End_ScatEdit.xlsx')

#Clean up times into YYYYMMDDHHMM
labels_start = np.squeeze(np.array(labels_start_raw), axis = 1)
labels_end = np.squeeze(np.array(labels_end_raw), axis = 1)

# Match Times with Modes

In [ ]:
#Array to save mode
mode = np.zeros(len(rad_time))-1

#Loop through all times with radar scans
for i in range(len(rad_time)):
    
    #Loop through all labelled periods to make sure the period has only one mode
    count = 0 
    for j in range(len(labels)):
        if labels_start[j] < rad_time[i] < labels_end[j]:
            count = count + 1
        
    #If period has one mode find it and save it
    if count == 1:
        for j in range(len(labels)): 
            if labels_start[j] < rad_time[i] < labels_end[j]:
                mode[i] = np.array(labels_num[j])

                
#Determine which indicies to keep (keep all rare modes and remove commong modes [BROAD and LLAP])
#Get indicies for all rare modes
ind_rare = np.squeeze(np.where((mode == 0)|(mode == 1)|(mode == 2)|(mode == 3)|(mode == 4)), axis = 0)
#ind_rare = np.squeeze(np.where((mode == 0)|(mode == 1)|(mode == 2)), axis = 0)

#Get 1000 random indicies for LLAP1
ind_llap = np.squeeze(np.where(mode == 5), axis = 0)
rand_llap = np.random.randint(0, len(ind_llap), 1500)

#Get 1000 random indicies for BROAD                    
ind_broad = np.squeeze(np.where(mode == 6), axis = 0)
rand_broad = np.random.randint(0, len(ind_broad), 1500)

#Combine indicies                      
ind = np.concatenate((ind_rare, ind_llap[rand_llap], ind_broad[rand_broad]), axis = 0)

#Load in data with proper indicies
mode_clean = np.array(mode[ind]) #Only use mode data with those indicies
%time ref_clean = np.array(ds_rad.bref[ind,:,:]) #Load in ref data for those indicies
ref_clean[np.isnan(ref_clean)] = 0 #Remove nans (NN cant ingest them) 

#Look at how many examples for each mode
print(np.count_nonzero(mode_clean == 0))
print(np.count_nonzero(mode_clean == 1))
print(np.count_nonzero(mode_clean == 2))
print(np.count_nonzero(mode_clean == 3))
print(np.count_nonzero(mode_clean == 4))
print(np.count_nonzero(mode_clean == 5))
print(np.count_nonzero(mode_clean == 6))

# Neural Net

In [ ]:
import tensorflow as tf
print(tf.__version__)
tf.keras.backend.clear_session()
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
keras.backend.clear_session()


#Separate into train and test data
train_per = 0.80 #Portion of data for training
num_train = np.int(len(mode_clean)*train_per)

indices = np.random.permutation(mode_clean.shape[0])
train_ind, test_ind = indices[:num_train], indices[num_train:]
x_train, x_test = ref_clean[train_ind,:,:], ref_clean[test_ind,:,:]
y_train, y_test = mode_clean[train_ind], mode_clean[test_ind]


#Normalize
x_train = keras.utils.normalize(x_train)
x_test = keras.utils.normalize(x_test)

In [ ]:
# input image dimensions
img_rows, img_cols = len(ref_clean[0,:,0]), len(ref_clean[0,0,:])
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols,1)
x_test = x_test.reshape(x_test.shape[0],img_rows, img_cols,1)
input_shape = (img_rows, img_cols,1)

from keras.utils import to_categorical
y_train2 = to_categorical(y_train)
y_test2 = to_categorical(y_test)


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(300, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(300, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(7, activation=tf.nn.softmax))

model.compile(optimizer = 'adam',
             loss ='sparse_categorical_crossentropy',
             metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])

model.fit(x_train, y_train, 
          batch_size = 10, 
          epochs = 20)

##########################################


# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape = input_shape))
# model.add(tf.keras.layers.MaxPooling2D((2, 2)))
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(300, activation='relu', kernel_initializer='he_uniform'))
# model.add(tf.keras.layers.Dropout(0.5))
# model.add(tf.keras.layers.Dense(300, activation='relu', kernel_initializer='he_uniform'))
# model.add(tf.keras.layers.Dropout(0.5))
# model.add(tf.keras.layers.Dense(300, activation='relu', kernel_initializer='he_uniform'))
# model.add(tf.keras.layers.Dropout(0.5))
# model.add(tf.keras.layers.Dense(7, activation='softmax'))
# # compile model
# opt = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9)
# model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
# model.fit(x_train, y_train2, 
#           batch_size = 10, 
#           epochs = 10)


###########################################

# model = Sequential()

# model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu', input_shape = input_shape))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))


# model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu'))
# model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
# model.add(Dropout(0.25))


# model.add(Flatten())
# model.add(Dense(256, activation = "relu"))
# model.add(Dropout(0.5))
# model.add(Dense(256, activation = "relu"))
# model.add(Dropout(0.5))
# model.add(Dense(7, activation = "softmax"))

# # compile model
# opt = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9)
# model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
# model.fit(x_train, y_train2, 
#           batch_size = 10, 
#           epochs = 10)

In [ ]:
print('\n# Evaluate on test data')
results = model.evaluate(x_test, y_test, batch_size=10, verbose=0)
print('test loss, test acc:', results)

print('\n# Generate predictions for 3 samples')
predictions = model.predict(x_test[:3])
print('predictions shape:', predictions.shape)

In [ ]:
#for x in np.arange(100):
for x in np.arange(500):


    predictions = model.predict(x_test[x:x+1,:,:])
    #print(predictions)

    num_predict = np.argmax(predictions)
    #print(predictions)
    print(np.max(predictions)*100)
    ind = test_ind[x]

    num_actual = np.int(mode_clean[ind])

    mode_labs = ['SHORE', 'MISC', 'OROG', 'HYBRID',  'MCV', 'LLAP1', 'BROAD',]

    print(x)
    print('predicted mode:{0}'.format(mode_labs[num_predict]))
    print('actual mode:{0}\n'.format(mode_labs[num_actual]))
    
    #plt.contourf(ref_clean[ind], levels = np.arange(0.5,30,2),cmap = cm.jet)
    plt.show()

# Plot

In [ ]:
import cartopy
import cartopy.crs as ccrs
from cartopy.io.srtm import srtm_composite
import cartopy.feature as cfeature
sys.path.append('/glade/u/home/tomg/modules')
import nclcmaps
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker



#Get lat and lons
lat = ds_rad.lat.values
lon = ds_rad.lon.values

for i in range(6,7):


    fig = plt.figure(num=None, figsize=(11,6), facecolor='w', edgecolor='k')
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.72, top=0.98, wspace=0.2, hspace=0.1)


    #Levels for theta
    tlmin = 5
    tlmax = 40.001
    tlevels = np.arange(tlmin,tlmax, 0.5)
    tlevels_ticks = np.arange(tlmin,tlmax,5)
    tlevels_ticks_labels = np.arange(tlmin,tlmax, 5).astype(int)


    #Colormap
    colors1 = np.array(nclcmaps.colors['prcp_1'])
    colors_int = colors1.astype(int)
    colors = list(colors_int)
    cmap_1 = nclcmaps.make_cmap(colors, bit=True)

    #Bounds
    xtick_freq = 120
    ytick_freq = 120
    



    #########################################################################
    ####################### 1. Tug Hill Domain ##############################
    #########################################################################

    ax = plt.subplot(111,aspect = 'equal',projection=ccrs.Mercator(globe = None, central_longitude=180))


    land = cartopy.feature.LAND.with_scale('10m')
    lakes = cartopy.feature.LAKES.with_scale('10m')
    coast = cartopy.feature.COASTLINE.with_scale('10m')
    bord = cartopy.feature.BORDERS.with_scale('10m')
    
    ax.add_feature(land)
    ax.add_feature(lakes, edgecolor = 'k', zorder = 5)
    ax.add_feature(coast, zorder = 5)
    ax.add_feature(bord, zorder = 5)

    lat2 = np.rot90(np.tile(lat,(len(lon),1)),3)
    lon2 = np.rot90(np.tile(lon,(len(lat),1)),0)-180
    proj = ccrs.Mercator()
    out_points = proj.transform_points(ccrs.PlateCarree(), 
                            lon2, lat2)
    
    
    #Make prediction
    predictions = model.predict(x_test[i:i+1,:,:])
    #Index of prediction
    num_predict = np.argmax(predictions)
    #Index of actual
    ind = test_ind[i]
    num_actual = np.int(mode_clean[ind])


    
    #Plot
    bref_plot = plt.contourf(out_points[:,:, 0], out_points[:,:, 1], ref_clean[ind], tlevels, alpha = 1,
        extend = 'max', cmap = cmap_1, zorder = 6)


    ax.set_extent([-78, -74.5, 43, 44.5], crs=ccrs.PlateCarree())


    #Plot Characteristics
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    plt.ylabel('Latitude', fontsize = 12)
    plt.xlabel('Longitude', fontsize = 12)


    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=2, color='gray', alpha=0.5, linestyle='--')
    
    gl.xlocator = mticker.FixedLocator(np.arange(-180,180,0.6))
    gl.ylocator = mticker.FixedLocator(np.arange(-180,180,0.3))
    gl.yformatter = LATITUDE_FORMATTER
    gl.xformatter = LONGITUDE_FORMATTER
    gl.ylabel_style = {'size': 13}
    gl.xlabel_style = {'size': 13}
    
    #Classificaiton    #Print mode corresponding to index
    mode_labs = ['SHORE', 'MISC', 'OROG', 'HYBRID',  'MCV', 'LLAP', 'BROAD',]
    props = dict(boxstyle='square', facecolor='white', alpha=0.8, ec="gray")
    ax.text(0.03, 0.76, 'Deep Neural Net mode: {}\nConfidence: {:.2f}%\nActual mode: {}'.format(mode_labs[num_predict],np.max(predictions)*100,mode_labs[num_actual]),\
        transform=ax.transAxes,bbox = props, fontsize = 17, zorder = 20)

    #Colorbar
    cbaxes = fig.add_axes([0.83, 0.323, 0.04, 0.5])
    cbar = plt.colorbar(bref_plot, orientation='vertical', cax = cbaxes, ticks = tlevels_ticks)
    cbar.ax.set_yticklabels(tlevels_ticks_labels)
    cbar.ax.tick_params(labelsize=14)
    plt.text(0.63,-0.14, 'Reflectivity\n(dBZ)', fontsize = 15, ha='center', va='center')

    #Plot and Save Figure
    plt.savefig("tug_ml_classify_examples_{}.png".format(i), dpi = 300)
    plt.show(fig)
    
    

    
    